# Tutoriel - Word2vec en utilisant Pytorch

Ce notebook explique comment implémenter la technique de NLP, appelée word2vec, à l’aide de Pytorch. Word2vec a pour objectif principal de construire un "word embedding", c’est-à-dire une représentation de mots ("latent and semantic free") dans un espace continu. Pour ce faire, cette approche exploite un réseau de neurones peu profond, avec seulement 2 couches. Ce tutoriel explique : 

* comment générer l'ensemble de données adapté à word2vec
* comment construire le réseau de neurones
* comment accélérer l'approche


## Les données

Présentons les concepts de base du NLP : 

* Corpus : le corpus est la collection de textes définissant le jeu de données
* vocabulaire: l'ensemble des mots contenu dans les données

En guise d'exemple, nous utilisons le nouveau corpus contenu dans la base de données "Brown", disponible dans la librairie nltk. 

In [99]:
# Téléchargement des différentes bibliothèques... A faire une fois
#nltk.download('universal_tagset') #Pour traduire les types de tags
#nltk.download('brown')
#nltk.download('words')
#nltk.download('stopwords')
#nltk.download('names')
#nltk.download('cmudict')

import re
import nltk
from nltk.corpus import brown


corpus = []

### Lexiques et listes de mots dans nltk

Le package NLTK comprend également un certain nombre de lexiques et de listes de mots. Celles-ci sont accessibles comme des corpus de texte. Les exemples suivants illustrent l’utilisation des corpus de la liste de mots:

In [92]:
from nltk.corpus import names, stopwords, words

print(words.fileids())
print(words.words('en')[1:5]) 
print(len(words.words('en')))
print(stopwords.fileids()[1:5])
print(stopwords.words('french')[1:5])
print(names.fileids())
print(names.words('female.txt')[1:5])

['en', 'en-basic']
['a', 'aa', 'aal', 'aalii']
235886
['azerbaijani', 'danish', 'dutch', 'english']
['aux', 'avec', 'ce', 'ces']
['female.txt', 'male.txt']
['Abagail', 'Abbe', 'Abbey', 'Abbi']


Le corpus du dictionnaire de prononciation CMU contient des transcriptions de plus de 100 000 mots. Vous pouvez y accéder sous forme de liste d'entrées (chaque entrée étant composée d'un mot, d'un identifiant et d'une transcription) ou sous forme de dictionnaire de mots en listes de transcriptions. Les transcriptions sont codées sous forme de n-uplets de chaînes de phonèmes.


In [98]:
from nltk.corpus import cmudict
print(cmudict.entries()[653]) 
# charger le corpus entier cmudict dans le dictionnaire python:
transcr = cmudict.dict()
print([transcr[w][0] for w in 'Kim Antunez'.lower().split()])

('acetate', ['AE1', 'S', 'AH0', 'T', 'EY2', 'T'])
[['K', 'IH1', 'M'], ['AA0', 'N', 'T', 'UW1', 'N', 'EH0', 'Z']]


### Appropriation des données de type corpus

Commençons par nous approprier le fonctionnement du format "corpus" dans Python...


In [44]:
# Description du corpus brown
print(brown.readme())

BROWN CORPUS

A Standard Corpus of Present-Day Edited American
English, for use with Digital Computers.

by W. N. Francis and H. Kucera (1964)
Department of Linguistics, Brown University
Providence, Rhode Island, USA

Revised 1971, Revised and Amplified 1979

http://www.hit.uib.no/icame/brown/bcm.html

Distributed with the permission of the copyright holder,
redistribution permitted.



In [34]:
print("Chemin de la base de données Brown : ")
print(str(nltk.corpus.brown).replace('\\\\','/'))

print("Nombre de mots du fichier ca01 la base Brown ")
len(brown.words('ca01'))

print("\n 10 premiers noms de fichiers de la base Brown : ")
print(brown.fileids()[:10])

print("\n 100 premiers caractères du fichier ca01 la base Brown : ")
print(brown.raw('ca01')[:100])

print("\n 10 premiers mots du fichier ca01 la base Brown : ")
print(brown.words('ca01')[1:10])

#print("\n Première phrase du fichier ca01 la base Brown : ")
#print(brown.sents('ca01')[1:10])

#print("\n Premier paragraphe du fichier ca01 la base Brown : ")
#print(brown.paras('ca01'))

print("\n Mots de la base Brown en entier : ")
print(brown.words())

Chemin de la base de données Brown : 
<CategorizedTaggedCorpusReader in 'W:/AppData/nltk_data/corpora/brown'>

 10 premiers noms de fichiers de la base Brown : 
['ca01', 'ca02', 'ca03', 'ca04', 'ca05', 'ca06', 'ca07', 'ca08', 'ca09', 'ca10']

 100 premiers caractères du fichier ca01 la base Brown : 


	The/at Fulton/np-tl County/nn-tl Grand/jj-tl Jury/nn-tl said/vbd Friday/nr an/at investigation/nn

 10 premiers mots du fichier ca01 la base Brown : 
['Fulton', 'County', 'Grand', 'Jury', 'said', 'Friday', 'an', 'investigation', 'of']

 Mots de la base Brown en entier : 
['The', 'Fulton', 'County', 'Grand', 'Jury', 'said', ...]


La base Brown a la particularité d'être annotée (pas seulement en texte plein). Elle est annotée avec des balises de partie de parole et définit des méthodes supplémentaires étiquetées_*(), dans lesquels les mots sont des nuplets (mot, balise), plutôt que de simples chaînes de mots. Ici les tags semblent correspondre au type des mots

In [54]:
print(brown.tagged_words())
print(brown.tagged_words(tagset='universal')) 


[('The', 'AT'), ('Fulton', 'NP-TL'), ...]
[('The', 'DET'), ('Fulton', 'NOUN'), ...]


Plusieurs corpus inclus dans NLTK contiennent des documents classés par sujet, genre, polarité, etc. Outre l’interface de corpus standard, ces corpus permettent d’accéder à la liste des catégories et à l'association entre les documents et leurs catégories (dans les deux sens). On peut accéder aux catégories à l'aide de la méthode categories(). Cette méthode a un argument facultatif qui spécifie un document ou une liste de documents, nous permettant d'associer (un ou plusieurs) documents vers (une ou plusieurs) catégories.

Outre la mise en correspondance des catégories et des documents, ces corpus permettent un accès direct à leur contenu via les catégories. Au lieu d'accéder à un sous-ensemble d'un corpus en spécifiant un ou plusieurs ID de fichier, nous pouvons identifier une ou plusieurs catégories, ici la catégorie news. Notez que qu'il le faut pas préciser à la fois les documents et catégories, sinon cela renverait une erreur.

In [104]:
print(brown.categories()[1:5]) 
print(brown.categories(['ca01','cb01']))
print(brown.tagged_words(categories='news'))

#Dans le contexte d'un système de catégorisation de texte,
#nous pouvons facilement tester si la catégorie attribuée à un document est correcte :
def classify(doc): return 'news'   #
doc = 'ca01'
print(classify(doc) in brown.categories(doc))


['belles_lettres', 'editorial', 'fiction', 'government']
['editorial', 'news']
[('The', 'AT'), ('Fulton', 'NP-TL'), ...]
True


### Préparations de nos données dans brown

Revenons au traitement de la base brown pour le NLP. 

Ici, les caractères autres que des lettres sont supprimés de la chaîne. De plus, le texte est mis en minuscule.


In [137]:
#print(brown.fileids('news')[1:5])
#print(brown.sents('ca01')[0][1:5]) #5 premiers mots de la 1e phrase
#raw_text = list(itertools.chain.from_iterable(brown.sents('ca01')))[1:5] #idem sans préciser de numéro de phrase
#print(' '.join(raw_text))

import itertools  #NEW KIM

for cat in ['news']:  #On se restreint à la catégorie "news" des articles contenus dans brown. 
    for text_id in brown.fileids(cat): #On parcourt tous les fichiers de la catégorie news
        raw_text = list(itertools.chain.from_iterable(brown.sents(text_id))) #on récupère les mots
        text = ' '.join(raw_text) #justaxposer les mots en mettant des espaces
        text = text.lower() #texte mis en minuscule. 
        text.replace('\n', ' ') #on supprime les sauts de lignes
        text = re.sub('[^a-z ]+', '', text) #on enlève les caractères non alphabétiques
        corpus.append([w for w in text.split() if w != '']) #on intègre ces mots sous forme de 
        # liste de liste dans corpus

print(corpus[0][1:10]) #Les 10 premiers mots du premier texte (ca01) de corpus

['fulton', 'county', 'grand', 'jury', 'said', 'friday', 'an', 'investigation', 'of']


### Sous-échantillonnage des mots fréquents

La première étape du prétraitement des données consiste à équilibrer les occurrences de mots dans les données. Pour ce faire, nous effectuons un sous-échantillonnage des mots fréquents. Appelons $p_i$ la proportion du mot i dans le corpus. Alors la probabilité $P(wi)$ de garder le mot dans le corpus est définie comme suit :



$$P(w_i) = \dfrac{10^{-3}}{p_i}\left(\sqrt{10^3 p_i} + 1\right)$$

<span style="background-color: #FFFF00">Formule à comprendre</span>

In [155]:
from collections import Counter
import random, math

# On fixe l'aléa pour bien pouvoir comprendre nos résultats
random.seed(1)

#Retourne un float aléatoire dans l'intervalle [0.0, 1.0).
#print(random.random())

# Fonction qui en entrée prend un corpus et renvoie un corpus filtré
def subsample_frequent_words(corpus): #création d'une fonction qui prend un corpus en entrée
    filtered_corpus = [] 
    word_counts = dict(Counter(list(itertools.chain.from_iterable(corpus)))) #{'the': 159650, 'fulton': 350, ...
    sum_word_counts = sum(list(word_counts.values())) #nombre total de mots
    word_counts = {word: word_counts[word]/float(sum_word_counts) for word in word_counts} 
    # calculs de proportion dont la somme fait 100 {'the': 0.07339892418739369, 'fulton': 0.00016091214197048412,
    for text in corpus: #on parcourt tous les articles du corpus
        filtered_corpus.append([]) #on crée une sous liste à chaque nouvel article
        for word in text: #et pour tous les mots de l'article
            if random.random() < (1+math.sqrt(word_counts[word] * 1e3)) * 1e-3 / float(word_counts[word]):
                #si une proportion tirée au hasard est inférieure à la probabilité de garder ce mot
                filtered_corpus[-1].append(word) #on enlève ce mot. 
    return filtered_corpus

<span style="background-color: #FFFF00"> La condition random < P(wi) ne me semble pas intuitive</span>

In [181]:
#On applique la fonction de filtrage au corpus
corpus = subsample_frequent_words(corpus)
vocabulary = set(itertools.chain.from_iterable(corpus)) # transforme le nouveau corpus au format
#{'bernadines', 'removed', 'lining',

word_to_index = {w: idx for (idx, w) in enumerate(vocabulary)} # on associe un index a chaque mot
#{'bernadines': 0, 'removed': 1,
index_to_word = {idx: w for (idx, w) in enumerate(vocabulary)} # on associe un mot a chaque index
#{0: 'bernadines', 1: 'removed',
 

<span style="background-color: #FFFF00"> Je ne comprends pas pourquoi vocabulary est réordonné </span>

### Construire des sacs de mots

Word2vec est une approche de "sac" de mots. Pour chaque mot de l'ensemble de données, nous devons extraire les mots de contexte (*context words*), c'est-à-dire les mots voisins dans une certaine fenêtre (*window*) de longueur fixe. Par exemple, dans la phrase suivante :

*My cat is lazy, it sleeps all day long*

Si nous considérons le mot cible (*target word*) *lazy* et que nous choisissons une fenêtre de taille 2, les mots du contexte sont *cat*, *is*, *it* et *sleeps*.

In [192]:
#Petit aparté pour comprendre la fonction enumerate de Python
l1 = ["eat","sleep","repeat"] 
s1 = "geek"
obj1 = enumerate(l1) 
obj2 = enumerate(s1)   
print("Return type:",type(obj1))
print(list(enumerate(l1))) #C'est plutôt cette fonction d'indiciation de mot qu'on utilise ci-dessous
print(list(enumerate(s1,2))) #Ici on compte le nombre de lettres dans un mot

Return type: <class 'enumerate'>
[(0, 'eat'), (1, 'sleep'), (2, 'repeat')]
[(2, 'g'), (3, 'e'), (4, 'e'), (5, 'k')]


In [202]:
import numpy as np

#print(list(enumerate(corpus[0]))[:10])
#[(0, 'fulton'), (1, 'county'),

context_tuple_list = []
w = 4 #taille de la fenêtre = 4

# créer des pairs de mots cibles et contexte
for text in corpus: #pour tous les articles du corpus (nettoyé par la méthodes ci-dessus)
    for i, word in enumerate(text): #pour chaque association nb de mots (i) + mot (word)
        first_context_word_index = max(0,i-w)
        last_context_word_index = min(i+w, len(text))
        for j in range(first_context_word_index, last_context_word_index):
            if i!=j:
                context_tuple_list.append((word, text[j]))
                
#print(context_tuple_list[0:10])    
#[('fulton', 'county'), ('fulton', 'grand'), ('fulton', 'jury'), ('county', 'fulton')...
print("Il y a {} paires de mots cibles et de contexte".format(len(context_tuple_list)))

Il y a 10702487 paires de mots cibles et de contexte


<span style="background-color: #FFFF00"> Tester les autres types de nettoyage ? Negative et pair sampling </span>

## Construire le réseau

Il existe deux approches pour word2vec : 

* CBOW (*Continuous Bag Of Words* ou sac continu de mots). Il prédit le mot cible conditionnellement au contexte. En d'autres termes, les mots de contexte sont l'entrée et le mot cible est la sortie.

* Skip-gram. Il prédit le contexte conditionnellement au mot cible. En d'autres termes, le mot cible est l'entrée et les mots de contexte sont la sortie.

Le code suivant concerne la méthode CBOW.

Le vocabulaire est représenté sous la forme d'un codage à une seule étape <span style="background-color: #FFFF00">(traduction?)</span> (*one hot encoding*), ce qui signifie que la variable d'entrée est un vecteur de la taille du vocabulaire (de taille n si n mots). Pour un mot, ce vecteur vaut 0 partout sauf à l'endroit de l'indice du mot dans le vocabulaire (i pour le ième mot et $x_i$ = 1).

Le codage à une étape est mappé <span style="background-color: #FFFF00">(traduction?)</span> (*mapped*)  sur un *embedding* <span style="background-color: #FFFF00">(traduction?)</span>, c'est-à-dire une représentation latente du mot en tant que vecteur contenant des valeurs continues et dont la taille est plus petit que le vecteur de codage *one-hot*.

Pour chaque mot de contexte, une fonction softmax prend l'*embedding du mot*, produisant une distribution de probabilité du mot cible sur le vocabulaire.

![alt text](https://rguigoures.github.io/images/cbow.png "Schéma de la méthode CBOW")



### Quelques précisions méthodologiques 

#### la fonction zero_grad

La fonction  `zero_grad` initialise les gradients de tous les paramètres du modèle à zéro.

Dans PyTorch, nous devons initialiser les gradients à 0 avant de commencer à effectuer une rétroprojection *backpropagation* <span style="background-color: #FFFF00">(= descente de gradient ?)</span> , car PyTorch accumule les gradients à chaque nouveau passage. Ceci est pratique quand on entraîne des RNN <span style="background-color: #FFFF00">(revoir RNN)</span>. Ainsi, l’action par défaut consiste à accumuler (c’est-à-dire à sommer) les gradients à chaque appel de `loss.backward()`.

De ce fait, lorsque vous démarrez votre boucle d’entraînement, vous devez idéalement mettre à zéro les gradients afin de mettre à jour le paramètre correctement. Sinon, le gradient indiquerait une direction autre que la direction souhaitée vers le minimum (ou le maximum, dans le cas d'objectifs de maximisation).

Voici un exemple pour comprendre : 

<span style="background-color: #FFFF00">sauf que j'ai rien compris => Comprendre avant à quoi correspond une descente de gradient ! </span>

In [261]:
import torch
from torch.autograd import Variable



x = Variable(torch.Tensor([[0]]), requires_grad=True)
print(x)
print(x.sin())
print(x.sin().backward())

for t in range(5):
    y = x.sin() 
    y.backward()
    
print(x.grad) # shows 5

#Calling x.grad.data.zero_() before y.backward() can make sure x.grad is exactly
#the same as current y’(x), not a sum of y’(x) in all previous iterations.

x = Variable(torch.Tensor([[0]]), requires_grad=True) 
print(x)

for t in range(5):
    if x.grad is not None:
        x.grad.data.zero_()
    y = x.sin() 
    y.backward()

print(x.grad) # shows 1

tensor([[0.]], requires_grad=True)
tensor([[0.]], grad_fn=<SinBackward>)
None
tensor([[6.]])
tensor([[0.]], requires_grad=True)
tensor([[1.]])
tensor([[0.]], grad_fn=<SinBackward>)


### la fonction Word2Vec

In [204]:
import torch
# import torch.nntorch.nn  as  nn
import torch.nn  as  nn #KIM
#import torch.autogradtorch.aut  as autograd
import torch.autograd  as  autograd #KIM
import torch.optim as optim
import torch.nn.functional as F


class Word2Vec(nn.Module):

    def __init__(self, embedding_size, vocab_size):
        super(Word2Vec, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_size)
        self.linear = nn.Linear(embedding_size, vocab_size)
        
    def forward(self, context_word):
        emb = self.embeddings(context_word)
        hidden = self.linear(emb)
        #out = F.log_softmax(hidden)
        out = F.log_softmax(hidden, dim=1)# KIM For matrices, it’s 1. For others, it’s 0.
        return out

### Arrêter l'algorithme avant la fin

Avant de commencer l’apprentissage, introduisons le concept d’arrêt précoce (*early stopping*). Il vise à arrêter l'apprentissage lorsque la perte (*loss*) ne diminue pas de manière significative (paramètre  `min_percent_gain`) après un certain nombre d'itérations (paramètre  `patience`). Un arrêt précoce est généralement utilisé sur la perte de validation (*validation loss*), mais dans le cas de word2vec, il n’y a pas de validation car l’approche n’est pas supervisée. Nous appliquons plutôt l'arrêt précoce sur les données d'entraînement à la place (*training loss*).

<span style="background-color: #FFFF00">mieux comprendre le concept de validation et d'apprentissage supervisé</span>


In [205]:
class EarlyStopping():
    def __init__(self, patience=5, min_percent_gain=0.1):
        self.patience = patience
        self.loss_list = []
        self.min_percent_gain = min_percent_gain / 100.
        
    def update_loss(self, loss):
        self.loss_list.append(loss)
        if len(self.loss_list) > self.patience:
            del self.loss_list[0]
    
    def stop_training(self):
        if len(self.loss_list) == 1:
            return False
        gain = (max(self.loss_list) - min(self.loss_list)) / max(self.loss_list)
        print("Loss gain: {}%".format(round(100*gain,2)))
        if gain < self.min_percent_gain:
            return True
        else:
            return False

### Apprentissage

Pour l'apprentissage (*learning*), nous utilisons l'entropie croisée (*cross entropy*) comme fonction de loss. Le réseau de neurones est entraîné avec les paramètres suivants:

* taille d'intégration : 200 (*embedding size*)
* taille du lot : 2000A (*batch size*)

In [227]:
## modifié car sinon trop long à tourner...
vocabulary_size = len(vocabulary) #12132
net = Word2Vec(embedding_size=2, vocab_size=vocabulary_size)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters())
#early_stopping = EarlyStopping()
early_stopping = EarlyStopping(patience=5, min_percent_gain=1)
context_tensor_list = []

## Etape 1 : On transforme les couple target context en couple de tensor (autre format)
for target, context in context_tuple_list[0:10]: #[('fulton', 'county'), ('fulton', 'grand')
#for target, context in context_tuple_list: 
    target_tensor = autograd.Variable(torch.LongTensor([word_to_index[target]])) #fulton devient tensor([6582])
    context_tensor = autograd.Variable(torch.LongTensor([word_to_index[context]]))  #county devient tensor([6582])
    context_tensor_list.append((target_tensor, context_tensor)) 
    #(tensor([6582]), tensor([8950])) ajouté à context_tensor_list

In [ ]:
## Etape 2 : 
while True:
    losses = []
    for target_tensor, context_tensor in context_tensor_list:
        net.zero_grad()
        log_probs = net(context_tensor)
        loss = loss_function(log_probs, target_tensor)
        loss.backward()
        optimizer.step()
        losses.append(loss.data)
    print("Loss: ", np.mean(losses))
    early_stopping.update_loss(np.mean(losses))
    if early_stopping.stop_training():
        break

## Speed up the approach

The implementation introduced is pretty slow. But good news, there are solutions for speeding up the computation.

### Batch learning

In order to speed up the learning, we propose to use batches. This implies that a bunch of observations are forwarded through the network before doing the backpropagation. Besides being faster, this is also a good way to regularize the parameters of the model.

In [61]:
import random

def get_batches(context_tuple_list, batch_size=100):
    random.shuffle(context_tuple_list)
    batches = []
    batch_target, batch_context, batch_negative = [], [], []
    for i in range(len(context_tuple_list)):
        batch_target.append(word_to_index[context_tuple_list[i][0]])
        batch_context.append(word_to_index[context_tuple_list[i][1]])
        batch_negative.append([word_to_index[w] for w in context_tuple_list[i][2]])
        if (i+1) % batch_size == 0 or i == len(context_tuple_list)-1:
            tensor_target = autograd.Variable(torch.from_numpy(np.array(batch_target)).long())
            tensor_context = autograd.Variable(torch.from_numpy(np.array(batch_context)).long())
            tensor_negative = autograd.Variable(torch.from_numpy(np.array(batch_negative)).long())
            batches.append((tensor_target, tensor_context, tensor_negative))
            batch_target, batch_context, batch_negative = [], [], []
    return batches

### Negative examples

The default word2vec algorithm exploits only positive examples and the output function is a softmax. However, using a softmax slows down the learning: softmax is normalized over all the vocabulary, then all the weights of the network are updated at each iteration. Consequently we decide using a sigmoid function as an output instead: only the weights involving the target word are updated. But then the network does not learn from negative examples anymore. That’s why we need to input artificially generated negative examples.

Once we have built the data for the positive examples, i.e the words in the neighborhood of the target word, we need to build a data set with negative examples. For each word in the corpus, the probability of sampling a negative context word is defined as follows:

$$P(w_i) = \dfrac{\mid w_i \mid^{\frac{3}{4}}}{\displaystyle\sum_{j=1}^n\mid w_j \mid^{\frac{3}{4}}}$$

In [81]:
from numpy.random import multinomial

def sample_negative(sample_size):
    sample_probability = {}
    word_counts = dict(Counter(list(itertools.chain.from_iterable(corpus))))
    normalizing_factor = sum([v**0.75 for v in word_counts.values()])
    for word in word_counts:
        sample_probability[word] = word_counts[word]**0.75 / normalizing_factor
    words = np.array(list(word_counts.keys()))
    while True:
        word_list = []
        sampled_index = np.array(multinomial(sample_size, list(sample_probability.values())))
        for index, count in enumerate(sampled_index):
            for _ in range(count):
                 word_list.append(words[index])
        yield word_list

In [82]:
## modifié car sinon trop long à tourner...

import numpy as np
context_tuple_list = []
#w = 4
w = 2
#negative_samples = sample_negative(8)
negative_samples = sample_negative(2)

for text in corpus[1:2]:
#for text in corpus:
    for i, word in enumerate(text):
        first_context_word_index = max(0,i-w)
        last_context_word_index = min(i+w, len(text))
        for j in range(first_context_word_index, last_context_word_index):
            if i!=j:
                context_tuple_list.append((word, text[j], next(negative_samples)))
print("There are {} pairs of target and context words".format(len(context_tuple_list)))

There are 4643 pairs of target and context words


### The network

The main difference from the network introduced above lies in the fact that we don’t need a probability distribution over words as an output anymore. We can instead have a probability for each word. To get that, we can replace the softmax out output by a sigmoid, taking values between 0 and 1.

The other main difference is that the loss needs to be computed on the observe output only, since we provide the expected output as well as a set of negative examples. To do so, we can use a negative logarithm of the output as a loss function.

For a target word $w_T$, a context word $w_C$ and a negative example $w_N$, respective embeddings are defined as $e_T$, $e_C$ and $e_N$. The loss function l is defined as follows:

$$l = -log(\sigma(e_T^T e_C)) - \displaystyle\sum_i log(\sigma(- e_T^T e_{N,i}))$$

In [83]:
import torch
import torch.nn as nn
import torch.autograd as autograd
import torch.optim as optim
import torch.nn.functional as F


class Word2Vec(nn.Module):

    def __init__(self, embedding_size, vocab_size):
        super(Word2Vec, self).__init__()
        self.embeddings_target = nn.Embedding(vocab_size, embedding_size)
        self.embeddings_context = nn.Embedding(vocab_size, embedding_size)

    def forward(self, target_word, context_word, negative_example):
        emb_target = self.embeddings_target(target_word)
        emb_context = self.embeddings_context(context_word)
        emb_product = torch.mul(emb_target, emb_context)
        emb_product = torch.sum(emb_product, dim=1)
        out = torch.sum(F.logsigmoid(emb_product))
        emb_negative = self.embeddings_context(negative_example)
        emb_product = torch.bmm(emb_negative, emb_target.unsqueeze(2))
        emb_product = torch.sum(emb_product, dim=1)
        out += torch.sum(F.logsigmoid(-emb_product))
        return -out

The neural network in trained with the following parameters: 

* embedding size: 200
* batch size: 2000

In [102]:
## modifié car sinon trop long à tourner...

import time

vocabulary_size = len(vocabulary)

loss_function = nn.CrossEntropyLoss()
#net = Word2Vec(embedding_size=200, vocab_size=vocabulary_size)
net = Word2Vec(embedding_size=10, vocab_size=vocabulary_size)
optimizer = optim.Adam(net.parameters())
early_stopping = EarlyStopping(patience=5, min_percent_gain=1)

while True:
    losses = []
    context_tuple_batches = get_batches(context_tuple_list, batch_size=2000)
    for i in range(len(context_tuple_batches)):
        net.zero_grad()
        target_tensor, context_tensor, negative_tensor = context_tuple_batches[i]
        loss = net(target_tensor, context_tensor, negative_tensor)
        loss.backward()
        optimizer.step()
        losses.append(loss.data)
    print("Loss: ", np.mean(losses))
    early_stopping.update_loss(np.mean(losses))
    if early_stopping.stop_training():
        break

Loss:  6034.86
Loss:  5976.067
Loss gain: 0.97%


Once the network trained, we can use the word embedding and compute the similarity between words. The following function computes the top n closest words for a given word. The similarity used is the cosine.

In [103]:
import numpy as np

def get_closest_word(word, topn=5):
    word_distance = []
    emb = net.embeddings_target
    pdist = nn.PairwiseDistance()
    i = word_to_index[word]
    lookup_tensor_i = torch.tensor([i], dtype=torch.long)
    v_i = emb(lookup_tensor_i)
    for j in range(len(vocabulary)):
        if j != i:
            lookup_tensor_j = torch.tensor([j], dtype=torch.long)
            v_j = emb(lookup_tensor_j)
            word_distance.append((index_to_word[j], float(pdist(v_i, v_j))))
    word_distance.sort(key=lambda x: x[1])
    return word_distance[:topn]

In [105]:
get_closest_word("mother")

[('fastgrossing', 2.5288169384002686),
 ('vienna', 2.7083771228790283),
 ('before', 2.883903741836548),
 ('kowalskis', 2.9128899574279785),
 ('loosely', 2.92732834815979)]